In [ ]:

distance = mature_biomass_1k.fastDistanceTransform().sqrt() \
    .multiply(ee.Image.pixelArea().sqrt()).rename("distance") \
    .updateMask(age_agbd.select("age")).float().clip(roi)


# Function to buffer and calculate mean
def buffer_and_calculate_mean(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(30), 1))
    mean_value = mature_biomass.reduceRegion(ee.Reducer.mean().unweighted(), buffered.geometry(), 100, \
                                               maxPixels=1e13)
    return buffered.set(mean_value)

pts_dist = distance.reduceRegions(unified_data_fc, ee.Reducer.first().setOutputs(["distance"]))

pts = pts_dist.map(buffer_and_calculate_mean)




In [ ]:
# Create the export task
task = ee.batch.Export.image.toAsset(
    image=distance,
    description="distance",
    assetId="projects/amazon-forest-regrowth/assets/distance_eucl_amaz_60000",
    region = distance.geometry(),
    crs="EPSG:4326",
    scale = 1000,
    maxPixels=5e10
)

# # Start the export task
task.start()


In [ ]:
age_agbd = ee.Image(f"{data_folder}/age_agbd")
distance = ee.Image(f"{data_folder}/distance_1000m_{region}")

# Sample the image to extract all points
points = age_agbd.sample(
    region=region_shp,
    scale=30,  # Adjust the scale as needed
    numPixels=1e13,  # Large number to ensure all points are sampled
    geometries=True  # Include geometries in the output
)

pts_dist = distance.reduceRegions(points, ee.Reducer.first().setOutputs(["distance"]), scale = 1000)

# pts_dist = pts_dist.map(lambda feature: ee.Feature(None, {'distance': feature.get('distance')}))

# Function to buffer and calculate mean
def buffer_and_calculate_mean(f):
    distance = ee.Number(f.get('distance'))
    buffered = ee.Feature(f.buffer(distance.add(1000), 1))
    mean_value = mature_biomass_aggregated.reduceRegion(ee.Reducer.mean().unweighted(), \
                                                        buffered.geometry(), 1000, maxPixels=1e13)
    return buffered.set(mean_value)

pts = pts_dist.map(buffer_and_calculate_mean)

pts_image = pts.reduceToImage(properties=['distance'], reducer=ee.Reducer.first())

export_task = ee.batch.Export.image.toAsset(
    image=pts_image,
    assetId=f"projects/amazon-forest-regrowth/assets/nearest_distance_1000m_countrywide_gee",
    description='nearest_distance_1000m_countrywide_gee',
    region = region_shp,
    scale=1000,
    maxPixels=1e13
)
export_task.start()
